In [ ]:
import numpy as np
import pandas as pd
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
import os
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_scolore
from tqdm import tqdm
from sklearn.decomposition import PCA
import seaborn as sns
from datetime import timedelta

### Use Cases Clustering

In [ ]:
# read CSV with grouped ocurrences

# method can be euclidean or dtw
method = 'euclidean'

path = '../output/temporal_UCs_pivot_seg.csv'

data = pd.read_csv(path, delimiter=',')

n_rows, n_cols = data.shape
file_name = os.path.basename(path)
print(f'File reading "{file_name}" finished, {n_rows} rows and {n_cols} columns')

In [ ]:
# remove datetime_id column from DataFrame so it doesn't show up in the plots
data = data.drop(['datetime_id'], axis=1)
print('datetime_id column removed from DataFrame')

n_rows, n_cols = data.shape
print(f'DataFrame: {n_rows} rows and {n_cols} columns')

#### Dataset pre-processing

In [ ]:
# print mean and standard deviation
mean = np.mean(data)
standard_deviation = np.std(data)

print('Mean:')
print(mean)

print('Standard deviation:')
print(standard_deviation)

In [ ]:
# standardize with mean = 0.0 and std = 0.0
scaler = TimeSeriesScalerMeanVariance(mu=0.0, std=1.0)
print('Starting data standardization...')
data = scaler.fit_transform(data)
print('Standardization finished\n')

mean = np.mean(data)
standard_deviation = np.std(data)
print(f'Mean: {mean}')
print(f'Standard deviation: {standard_deviation}')

#### Dimensionality reduction

In [ ]:
# PCA
# número of components, starts with 2 and increases iteratively
n_cols_pca = 2

# transform DataFrame to format compatible with PCA and silhouette_score
data_2d = data.reshape(data.shape[0], -1)

# define least amount of components where preserved initial data >= 90%
while True:
    pca = PCA(n_components=n_cols_pca)
    pca.fit(data_2d)
    percentage_preserved_pca = sum(pca.explained_variance_ratio_) * 100

    if percentage_preserved_pca >= 90.0:
        break
    else:
        n_cols_pca += 1

print(f'Least amount of columns to preserve >= 90% of initial dataset information: {n_cols_pca}')

pca = PCA(n_components=n_cols_pca)
print(f'\Starting dimensionality reduction from {n_cols} to {n_cols_pca} columns with PCA...')
pca = pca.fit(data_2d)
data = pca.transform(data_2d)

# if fit_tranform is used, it is not possible to use explained_variance_ratio_
# data = pca.fit_transform(data_std)

# percentage preserved after PCA
percentage_preserved_pca = sum(pca.explained_variance_ratio_) * 100
print('%5.2f%% of initial dataset information preserved after dimensionality reduction with PCA' % percentage_preserved_pca)

print('\nSample of first dataset row after PCA')
print(data[0])

# convert dataset to DataFrame
pca_cols_names = []

for i in range(0, n_cols_pca):
    pca_cols_names.append('component' + str(i+1))

data = pd.DataFrame(data, columns=pca_cols_names)

#### Define number of clusters for K-Means

In [ ]:
# Elbow method plot to define the number of clusters

# list with inertia values
inertia_list = []
# list with silhouette values
silhouette_list = []

# uncomment to hard-code desired method to euclidean or dtw
# method = 'euclidean'

# uncomment if PCA cell was not executed
# data_2d = data.reshape(data.shape[0], -1)

# define range to test K-Means
# range_clusters = range(2, 15)
range_clusters = range(2, 10) # 2 to 9 (used in the research paper)

print(f'Number of clusters training ({method})\n')
for n_clusters in range_clusters:
    if method == 'euclidean':
        model = TimeSeriesKMeans(n_clusters=n_clusters, metric='euclidean', max_iter=50, n_jobs=-1, random_state=0)
    elif method == 'dtw':
        model = TimeSeriesKMeans(n_clusters=n_clusters,
                                 metric='dtw',
                                 max_iter=50,
                                 max_iter_barycenter=100,
                                 n_jobs=-1,
                                 random_state=0
                                )
        
    with tqdm(total=1, desc=f'Training K-Means | Number of clusters = {n_clusters} | Method = {method}', dynamic_ncols=True) as pbar_kmeans:
        model.fit(data)
        pbar_kmeans.update(1)
    
    inertia_list.append(model.inertia_)
    training_labels = model.labels_

    # it is possible to calculate Silhouette only if the number of clusters is > 1
    if len(np.unique(training_labels)) > 1:
        with tqdm(total=1, desc=f'Silhouette calculation | Number of clusters = {n_clusters}', dynamic_ncols=True) as pbar:
            silhouette_kmeans_elbow = silhouette_score(data_2d, training_labels)
            pbar.set_postfix({'Silhouette': silhouette_kmeans_elbow})
            pbar.update()
            silhouette_list.append(silhouette_kmeans_elbow)

print('')
#print(inertia)

# number of clusters = max valor de Silhouette in list (list index +2 because if starts from 2 (clusters) to len -1)
n_clusters = silhouette_list.index(max(silhouette_list)) + 2
print(f'\nBest number de clusters (k): {n_clusters}')

plt.figure(figsize=(10, 5))
plt.plot(range_clusters, inertia_list, marker='o', linestyle='-')
plt.title(f'Elbow method to choose the k number of clusters ({method})')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.grid()
plt.show()

In [ ]:
# transform values in silhouette list to a dictionary, key = k clusters (starts with 2)
dic_silhouette = {}

for i, silhouette in enumerate(silhouette_list):
    dic_silhouette[i+2] = silhouette

dic_silhouette = {key: round(value, 2) for key, value in dic_silhouette.items()}
for key, val in dic_silhouette.items():
    print(f'{key}: {val}')

#### Clustering with K-Means

In [ ]:
# execução do K-Means para output no CSV
# uncomment to change method
# method = 'dtw'

# comment if cell above was executed
n_clusters = 8

# uncomment if PCA call was not executed
# data_2d = data.reshape(data.shape[0], -1)

def kmeans_fit(n_clusters, method, model, data, i):
    with tqdm(total=1, desc=f'Training K-Means | execution {i} | Number of clusters = {n_clusters} | Method = {method}', dynamic_ncols=True) as pbar_kmeans:
        model.fit(data)
        pbar_kmeans.update(1)
    labels = model.labels_
    inertia_kmeans = model.inertia_
    return labels, inertia_kmeans

def silhouette(n_clusters, labels, data):
    with tqdm(total=1, desc=f'Silhouette calculation | Number of clusters = {n_clusters}', dynamic_ncols=True) as pbar:
        silhouette_kmeans = silhouette_score(data, labels)
        pbar.set_postfix({'Silhouette': silhouette_kmeans})
        pbar.update(1)
    return silhouette_kmeans

print(f'K-Means ({method}) with {n_clusters} clusters\n')

# run 20 times with random_state not defined, use best result
final_labels = []
final_inertia = 0
final_silhouette = 0

for i in range(1, 21):
    if method == 'euclidean':
        kmeans_euc = TimeSeriesKMeans(n_clusters=n_clusters,
                                    metric='euclidean',
                                    max_iter=50,
                                    n_jobs=-1
                                    )
        # fit model
        labels_euc, inertia_kmeans_euc = kmeans_fit(n_clusters, method, kmeans_euc, data, i)
        print(f'Inertia: {inertia_kmeans_euc}')
        # calculate silhouette score
        if len(np.unique(labels_euc)) > 1:
            silhouette_kmeans_euc = silhouette(n_clusters, labels_euc, data)
        # if it was the best execution so far
        if silhouette_kmeans_euc > final_silhouette:
            final_labels = labels_euc
            final_silhouette = silhouette_kmeans_euc
            final_inertia = inertia_kmeans_euc
    
    if method == 'dtw':
        kmeans_dtw = TimeSeriesKMeans(n_clusters=n_clusters, 
                                    metric='dtw',
                                    max_iter=50,
                                    max_iter_barycenter=100,
                                    n_jobs=-1 # -1 uses all CPU threads in paralell
                                    )
        # fit model
        labels_dtw, inertia_kmeans_dtw = kmeans_fit(n_clusters, method, kmeans_dtw, data, i)
        print(f'Inertia: {inertia_kmeans_dtw}')
        # calculate silhouette score
        if len(np.unique(labels_euc)) > 1:
            silhouette_kmeans_dtw = silhouette(n_clusters, labels_dtw, data_2d)
        # if it was the best execution so far
        if silhouette_kmeans_dtw > final_silhouette:
            final_labels = labels_dtw
            final_silhouette = silhouette_kmeans_dtw
            final_inertia = inertia_kmeans_dtw

print(f'\Best execution: Silhouette Score: {final_silhouette} | Inertia : {final_inertia}')

# print cluster of each row
# for i, label in enumerate(labels_euc):
#     print(f"Série {i}: Cluster {label}")

In [ ]:
# output to CSV with labels

# read grouped CSV file again
data = pd.read_csv(path, delimiter=',')

file_name = os.path.basename(path)
n_rows, n_cols = data.shape
print(f'File reading {file_name} finished, {n_rows} rows and {n_cols} columns')

# add column with cluster labels
data['cluster'] = final_labels

# output file
data.to_csv('../kmeans/kmeans_' + method + '_' +  str(n_clusters) + '_clusters.csv', index=False)
print('File kmeans_' + method + '_' + str(n_clusters) + '_clusters.csv created')

#### Data visualization

In [ ]:
# read CSV with clusters
path_cluster = '../kmeans/kmeans_euclidean_8_clusters.csv'
df_kmeans = pd.read_csv(path_cluster, delimiter=',')

n_rows, n_cols = df_kmeans.shape
file_name = os.path.basename(path_cluster)
print(f'File reading {file_name} finished, {n_rows} rows and {n_cols} columns')

# Number of clusters in file
n_clusters = df_kmeans['cluster'].nunique()
print(f'Number of clusters: {n_clusters}')

In [ ]:
# split DataFrame, with each one being of one cluster
list_df_kmeans = []

for i in range(0, n_clusters):
    list_df_kmeans.append(df_kmeans[df_kmeans['cluster'] == i])

print('List of DataFrames for each cluster created')

# view statistics of each DataFrame
# for df in list_df_kmeans:
#     print(df.describe())
#     print('-'*80)

#list_df_kmeans[3].describe()

In [ ]:
# filter DataFrames to keep only the UCs with mean >= 1
for i, df in enumerate(list_df_kmeans):
    filtered_ucs = ['datetime_id'] + [column for column in df.columns[1:-1] if df[column].mean() >= 1] + ['cluster']
    list_df_kmeans[i] = df[filtered_ucs]

print('DataFrames filtered, UCs with mean >= 1 kept')

# define DataFrame to view in the plots
cluster = 3

# view statistics of each cluster
# for df in list_df_kmeans:
#     print(df.describe())
#     print('-'*80)

# for df in list_df_kmeans:
#     print(df.info())
#     print('****'*50)

In [ ]:
# define colors for the UCs in the scatter plot
colors = sns.color_palette("hsv", len(list_df_kmeans[cluster].columns[1:-1]))
print(f'len colors: {len(colors)}')

# list of UCs correspondent to colors
columns = list_df_kmeans[cluster].columns[1:-1]
columns = columns.to_list()
print(f'len columns: {len(columns)}\n')

# dicitionary with the colors of each UC
map_colors = {column: color for column, color in zip(columns, colors)}

for key, value in map_colors.items():
    print(f'{key}: {value}')

In [ ]:
# add datetime_id_d column (rounded in days to use for filtering)
list_df_kmeans[cluster]['datetime_id_d'] = pd.to_datetime(list_df_kmeans[cluster]['datetime_id']).dt.strftime('%Y-%m-%d')
datetimes = list_df_kmeans[cluster]['datetime_id_d'].unique()
n_datetimes = list_df_kmeans[cluster]['datetime_id_d'].nunique()

print(f'{n_datetimes} datetimes in DataFrame')
print(datetimes)

In [ ]:
# discover the days which a certain UC ocurred the most
uc_search = 'l017_du'
map_ocurrences_per_day = {}

for idx, row in list_df_kmeans[cluster].iterrows():
    if row[uc_search] > 0:
        # extract day from datetime_id and format as '%Y-%m-%d'
        day = pd.to_datetime(row['datetime_id']).strftime('%Y-%m-%d')
    
        # check if the day is already in the map, if not, create an entry in the map for it
        if day not in map_ocurrences_per_day:
            map_ocurrences_per_day[day] = 0
        
        # add the ocorrence to the correspondent day in the map
        map_ocurrences_per_day[day] += row[uc_search]

for key, value in map_ocurrences_per_day.items():
    print(f'{key}: {value}')

# chosen day = item with most ocurrences of UC searched
day_search = max(map_ocurrences_per_day, key=map_ocurrences_per_day.get)
ocurrences = map_ocurrences_per_day[day_search]
print(f'\nDatetime with most ocurrences of UC {uc_search}: {day_search} with {ocurrences} ocurrences')

In [ ]:
# DO NOT RUN WITH CELL BELOW IS EXECUTED
# choose day to filter with the max mean of UCs ocurrences
day = ''
day_plus_1 = ''
max_percentages_sum = 0
day_filter = ''
day_plus_1_filter = ''

for i, row in enumerate(list_df_kmeans[cluster]):
    day = datetimes[i]
    day = pd.Timestamp(day)
    day_plus_1 = day + timedelta(days=1)
    day_plus_1 = day_plus_1.strftime('%Y-%m-%d')
    day = day.strftime('%Y-%m-%d')
    df_filtered = list_df_kmeans[cluster].query(f"'{day}' <= datetime_id < '{day_plus_1}'")
    # -2 para não considerar a coluna com o label do cluster
    positive_percentages = (df_filtered.iloc[:,1:-2] > 0).sum(axis=1) / (len(df_filtered.columns)-2) * 100
    sum_positive_percentages = 0
    for percentage in positive_percentages:
        sum_positive_percentages += percentage
        if sum_positive_percentages > max_percentages_sum:
            max_percentages_sum = sum_positive_percentages
            day_filter = day
            day_plus_1_filter = day_plus_1

day = day_filter
day_plus_1 = day_plus_1_filter

max_percentages_sum = round(max_percentages_sum, 2)
print(f'\Largest sum of UCs ocurrences percentages in {day}: {max_percentages_sum}')

In [ ]:
# DO NOT RUN IF CELL ABOVE WAS EXECUTED
# hard-code date range
day = '2023-03-01'
day = pd.Timestamp(day)

day_plus_1 = day + timedelta(days=1)
day_plus_1 = day_plus_1.strftime('%Y-%m-%d')

day = day.strftime('%Y-%m-%d')

print(f'Date range set between {day} and {day_plus_1}')

#### Scatter plot

In [ ]:
# scatter plot of cluster 3
plt.figure(figsize=(12, 6))

# extract corresponding colors and each column using map_colors
colors = [map_colors[col] for col in list_df_kmeans[cluster].columns[1:-2]]

# filter dataset to reduce its size
subset = list_df_kmeans[cluster].query(f"'{day}' <= datetime_id < '{day_plus_1}'")

# if number of datetimes in day > 50, filter hours range to 00:00-11:00 OR 11:15-23:45
if len(subset) > 50:
    subset = subset.iloc[0:50] # first 50 values
    #subset = subset.iloc[len(subset)-51:-1] # last 50 values
    #subset = subset.iloc[17:68] # hard-code hours range
##########################################################################################

# number of datetimes in subset
n_datetimes_subset = subset['datetime_id'].nunique()
print(f'Datetimes: {n_datetimes_subset}')

# first and last datetime in filtered day
data1 = subset.iloc[0]['datetime_id']
data2 = subset.iloc[-1]['datetime_id']

## set highlighted UCs to display only their dots and lines in the plot
uc1 = 'l017_du'
uc2 = 'l090'
#uc3 = 'l081'

values_ucs1 = subset[uc1]
values_ucs2 = subset[uc2]
#values_ucs3 = subset[uc3]

# set list of highlighted UCs
list_highlighted_ucs = [uc1, uc2]
#list_highlighted_ucs = [uc1, uc2, uc3]

##################################### UCs dots ###############################################
for i, col in enumerate(list_df_kmeans[cluster].columns[1:-2]):
    # display only the dots of highlighted UCs
    if col in list_highlighted_ucs:
        plt.scatter(subset['datetime_id'], subset[col], color=colors[i], label=col, alpha=0.7)

################################# selected UCs lines #########################################
plt.plot(subset['datetime_id'], values_ucs1, color=map_colors[uc1], label=uc1, linewidth=2)
plt.plot(subset['datetime_id'], values_ucs2, color=map_colors[uc2], label=uc2, linewidth=2)
#plt.plot(subset['datetime_id'], values_ucs3, color=map_colors[uc3], label=uc3, linewidth=2)
##############################################################################################

############### change plot title accordingly to highlighted UCs ######## ####################
plt.title(f'Use Cases - Cluster {cluster} on {data1[0:10]} (highlight between {uc1} and {uc2})')
#plt.title(f'Use Cases - Cluster {cluster} on {data1[0:10]} (highlight between {uc1}, {uc2} and {uc3})')
#plt.title(f'Use Cases - Cluster {cluster} on {data1[0:10]} (highlight between {uc1}/{uc2[-2:]} and {uc3})')
#plt.title(f'Use Cases - Cluster {cluster} on {data1[0:10]}')
#plt.title(f'Use Cases - Cluster {cluster} on {data1[0:10]} (highlight between {uc1} and {uc2}/{uc3[-2:]})')

plt.xlabel('Datetime')
plt.ylabel('UCs Ocurrences')

# create legends using the mapping between UCs names and colors
legend_labels = [plt.Line2D([0], [0], marker='o', color='w', label=f' {key}', markersize=10, markerfacecolor=val) for key, val in map_colors.items()]
plt.legend(handles=legend_labels, loc='upper left', bbox_to_anchor=(1, 1))

# rotate x axis labels
plt.xticks(rotation=90)
plt.show()